In [ ]:
# STEP 1: GPU check + mount Google Drive + define paths for EfficientNet (CNN-Aug)

# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# 2) Check GPU
import torch, os
print("CUDA available:", torch.cuda.is_available())

# 3) EDIT THESE PATHS if your folders are different
#    (Using your Celeb-DF example; change to any dataset you want)
VIDEOS_REAL_DIR = "/content/drive/My Drive/test dataset Celeb DF/real"
VIDEOS_FAKE_DIR = "/content/drive/My Drive/test dataset Celeb DF/fake"

# Where extracted frames will go (local, fast)
FRAMES_REAL_DIR = "/content/frames/effb4/real"
FRAMES_FAKE_DIR = "/content/frames/effb4/fake"

# Your EfficientNet-B4 weights (CNN-Aug)
WEIGHTS_PATH = "/content/drive/My Drive/DeepfakeBench_weights/effnb4_best.pth"

# Where to save results (tables/metrics)
RESULTS_DIR = "/content/drive/My Drive/deepfake_results/celebdf_effb4"

# Make sure folders exist
for p in [FRAMES_REAL_DIR, FRAMES_FAKE_DIR, RESULTS_DIR]:
    os.makedirs(p, exist_ok=True)

# Quick sanity checks
print("Weights found:", os.path.exists(WEIGHTS_PATH))
print("Real videos folder exists:", os.path.exists(VIDEOS_REAL_DIR))
print("Fake videos folder exists:", os.path.exists(VIDEOS_FAKE_DIR))
print("Results will be saved to:", RESULTS_DIR)


Mounted at /content/drive
CUDA available: True
Weights found: True
Real videos folder exists: True
Fake videos folder exists: True
Results will be saved to: /content/drive/My Drive/deepfake_results/celebdf_effb4


In [ ]:
# STEP 2: Extract N uniform frames per video (real & fake) → saves JPGs in FRAMES_*_DIR

import os, cv2, glob
from tqdm import tqdm

# How many frames to save per video (edit if you want more/less)
N_FRAMES_PER_VIDEO = 20
JPEG_QUALITY = 95  # 80–95 is usually fine

# --- Helpers ---
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def list_videos(folder):
    exts = (".mp4", ".avi", ".mov", ".mkv", ".webm", ".flv")
    return sorted([p for p in glob.glob(os.path.join(folder, "*")) if p.lower().endswith(exts)])

def video_stem(path):
    return os.path.splitext(os.path.basename(path))[0]

def extract_uniform_frames(video_path, out_dir, n_frames=20, jpeg_q=95):
    """Extract n_frames roughly uniformly across the video."""
    name = video_stem(video_path)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return 0

    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Fallback if frame count is unreliable
    if total <= 0:
        # Count by reading
        frames = []
        while True:
            ok, _ = cap.read()
            if not ok: break
            frames.append(1)
        total = len(frames)
        cap.release()
        cap = cv2.VideoCapture(video_path)

    if total == 0:
        cap.release()
        return 0

    # Choose indices uniformly
    if total <= n_frames:
        picks = list(range(total))
    else:
        step = total / float(n_frames)
        picks = [int(i * step) for i in range(n_frames)]

    saved = 0
    for i, idx in enumerate(picks):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame_bgr = cap.read()
        if not ok:
            continue
        # Save as RGB JPEG
        out_path = os.path.join(out_dir, f"{name}_frame{i:04d}.jpg")
        cv2.imwrite(out_path, frame_bgr, [int(cv2.IMWRITE_JPEG_QUALITY), jpeg_q])
        saved += 1

    cap.release()
    return saved

def extract_folder(in_dir, out_dir, n_frames=20, jpeg_q=95):
    ensure_dir(out_dir)
    vids = list_videos(in_dir)
    total_saved = 0
    for vp in tqdm(vids, desc=f"Extracting from {os.path.basename(in_dir)}"):
        total_saved += extract_uniform_frames(vp, out_dir, n_frames, jpeg_q)
    return len(vids), total_saved

# --- Run extraction for your real & fake folders from STEP 1 ---
ensure_dir(FRAMES_REAL_DIR); ensure_dir(FRAMES_FAKE_DIR)

n_real, saved_real = extract_folder(VIDEOS_REAL_DIR, FRAMES_REAL_DIR, N_FRAMES_PER_VIDEO, JPEG_QUALITY)
n_fake, saved_fake = extract_folder(VIDEOS_FAKE_DIR, FRAMES_FAKE_DIR, N_FRAMES_PER_VIDEO, JPEG_QUALITY)

print(f"\n✅ Frame extraction complete.")
print(f"Real videos: {n_real} → frames saved: {saved_real}")
print(f"Fake videos: {n_fake} → frames saved: {saved_fake}")
print(f"Frames saved to:\n  REAL: {FRAMES_REAL_DIR}\n  FAKE: {FRAMES_FAKE_DIR}")


Extracting from fake: 100%|██████████| 50/50 [01:45<00:00,  2.10s/it]


✅ Frame extraction complete.
Real videos: 50 → frames saved: 1000
Fake videos: 50 → frames saved: 1000
Frames saved to:
  REAL: /content/frames/effb4/real
  FAKE: /content/frames/effb4/fake


In [ ]:
# Copy local frames → Google Drive
DEST_BASE = "/content/drive/My Drive/frames/celebdf_effb4"
!mkdir -p "$DEST_BASE"
!rsync -ah --info=progress2 /content/frames/effb4/ "$DEST_BASE"/
print("Copied to:", DEST_BASE)


        129.50M 100%    3.78MB/s    0:00:32 (xfr#2000, to-chk=0/2003)
Copied to: /content/drive/My Drive/frames/celebdf_effb4


In [ ]:
# STEP 3 — EfficientNet-B4 (baseline) → AUC | EER | AP on Celeb-DF frames
import os, glob, re, sys, subprocess, numpy as np, pandas as pd
from PIL import Image
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as TF
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

# ---- Paths from Steps 1&2 (fallback defaults if needed) ----
FRAMES_REAL_DIR = globals().get("FRAMES_REAL_DIR", "/content/frames/effb4/real")
FRAMES_FAKE_DIR = globals().get("FRAMES_FAKE_DIR", "/content/frames/effb4/fake")
RESULTS_DIR     = globals().get("RESULTS_DIR", "/content/drive/My Drive/deepfake_results/celebdf_effb4")
os.makedirs(RESULTS_DIR, exist_ok=True)

# If your EfficientNet baseline has its own weights, set here:
WEIGHTS_PATH_EFFNET = globals().get("WEIGHTS_PATH", "/content/drive/My Drive/DeepfakeBench_weights/effnb4_best.pth")

# ---- Inference knobs ----
IMG_SIZE   = 380
BATCH_SIZE = 32
NUM_WORKERS = 0

TRY_TTA       = [False, True]
TRY_NORM      = ["no_norm", "imagenet"]
TRY_PREPROC   = ["short_center"]           # keep aspect + center-crop
TRY_CONF_FILT = [0.0, 0.2, 0.3]            # drop low-confidence frames
AGGS          = ("median", "perc80", "top10", "trim10")

# ---- EfficientNet dependency ----
def _pip_quiet(*pkgs):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", *pkgs],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
try:
    from efficientnet_pytorch import EfficientNet
except Exception:
    _pip_quiet("efficientnet-pytorch==0.7.1")
    from efficientnet_pytorch import EfficientNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# ---- Model (EfficientNet-B4 baseline head: 2 classes) ----
class EfficientNetB4Binary(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = EfficientNet.from_name('efficientnet-b4')
        self.backbone._fc = nn.Identity()
        self.head = nn.Linear(1792, 2)  # [real, fake]
    def forward(self, x):
        x = self.backbone(x)
        return self.head(x)

assert os.path.isfile(WEIGHTS_PATH_EFFNET), f"Weights not found: {WEIGHTS_PATH_EFFNET}"
model = EfficientNetB4Binary().to(device)
state = torch.load(WEIGHTS_PATH_EFFNET, map_location="cpu")
if isinstance(state, dict) and all(isinstance(k,str) for k in state.keys()):
    if all(k.startswith("module.") for k in state.keys()):
        state = {k.replace("module.","",1): v for k,v in state.items()}
model.load_state_dict(state, strict=False)
model.eval()
softmax = torch.nn.Softmax(dim=1)

# Quick probe for output shape (1 dummy)
with torch.no_grad():
    dummy = torch.zeros(1,3,IMG_SIZE,IMG_SIZE, device=device)
    out_shape = tuple(model(dummy).shape)
print(f"✅ EfficientNet-B4 loaded on {device.type}. Output shape: {out_shape}")

# ---- Data ----
IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
def is_img(p): return p.lower().endswith(IMG_EXTS)

def infer_video_name(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.split(r"_frame\d+$", stem)
    if len(m) > 1 and m[0]: return m[0]
    m2 = re.sub(r"[_\-]\d+$", "", stem)
    return m2 if m2 and m2 != stem else stem

def build_transform(norm, preproc):
    if preproc == "short_center":
        t_base = [transforms.Resize(IMG_SIZE), transforms.CenterCrop(IMG_SIZE)]
    else:
        t_base = [transforms.Resize((IMG_SIZE, IMG_SIZE))]
    t_norm = [] if norm == "no_norm" else [transforms.Normalize([0.485,0.456,0.406],
                                                                [0.229,0.224,0.225])]
    return transforms.Compose(t_base + [transforms.ToTensor()] + t_norm)

class FrameDataset(Dataset):
    def __init__(self, folders_labels, transform):
        files, labels = [], []
        for folder, lbl in folders_labels:
            f = sorted([p for p in glob.glob(os.path.join(folder, "*")) if is_img(p)])
            files += f; labels += [lbl]*len(f)
        self.files = files; self.labels = labels; self.t = transform
        if len(self.files)==0:
            raise RuntimeError(f"No images found under the provided frame folders.")
    def __len__(self): return len(self.files)
    def __getitem__(self, i):
        p = self.files[i]
        img = Image.open(p).convert("RGB")
        return self.t(img), self.labels[i], p, infer_video_name(p)

@torch.no_grad()
def score_frames(norm_kind="no_norm", tta=False, preproc="short_center"):
    t = build_transform(norm_kind, preproc)
    ds = FrameDataset([(FRAMES_REAL_DIR,0),(FRAMES_FAKE_DIR,1)], t)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=torch.cuda.is_available())
    probs, labels, paths, vnames = [], [], [], []
    for xb, yb, pb, vb in loader:
        xb = xb.to(device, non_blocking=True)
        logits = (model(xb) + model(TF.hflip(xb))) / 2 if tta else model(xb)
        p_fake = softmax(logits)[:,1].detach().cpu().numpy()
        probs.append(p_fake); labels.append(yb.numpy()); paths += list(pb); vnames += list(vb)
    probs = np.concatenate(probs); labels = np.concatenate(labels)
    return pd.DataFrame({"video_name": vnames,
                         "true_label": np.where(labels==1,"fake","real"),
                         "prob_fake": probs})

def video_metrics(scores, labels):
    auc = roc_auc_score(labels, scores)
    ap  = average_precision_score(labels, scores)
    fpr, tpr, thr = roc_curve(labels, scores); fnr = 1 - tpr
    i = int(np.nanargmin(np.abs(fnr - fpr)))
    eer = float((fpr[i] + fnr[i]) / 2.0)
    thr_eer = float(thr[i])
    return auc, eer, ap, thr_eer

def trimmed_mean(vals, trim=0.1):
    if len(vals)==0: return np.nan
    k = int(len(vals)*trim); vals = np.sort(vals)
    if k*2 >= len(vals): return float(np.mean(vals))
    return float(np.mean(vals[k:len(vals)-k]))

# ---- Score frames for a small grid, search best per-video AUC (tie: lower EER) ----
cache = {}
for norm in TRY_NORM:
    for tta in TRY_TTA:
        for pre in TRY_PREPROC:
            cache[(norm, tta, pre)] = score_frames(norm, tta, pre)

best = None  # (AUC, EER, AP, thr, desc, per_video_df)
for (norm, tta, pre), df in cache.items():
    # auto-flip orientation if it helps per-video avg AUC
    avg_df = df.groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
    y_avg  = (avg_df["true_label"]=="fake").astype(int).values
    s_avg  = avg_df["prob_fake"].values
    flip_needed = roc_auc_score(y_avg, 1 - s_avg) > roc_auc_score(y_avg, s_avg)
    df_use = df.copy()
    if flip_needed: df_use["prob_fake"] = 1 - df_use["prob_fake"]

    for filt in TRY_CONF_FILT:
        if filt > 0:
            df_f = df_use[np.abs(df_use["prob_fake"] - 0.5) >= filt].copy()
            missing = set(df_use["video_name"].unique()) - set(df_f["video_name"].unique())
            if missing:
                df_f = pd.concat([df_f, df_use[df_use["video_name"].isin(missing)]], ignore_index=True)
        else:
            df_f = df_use

        grouped = df_f.groupby(["video_name","true_label"])["prob_fake"]

        # median
        med = grouped.median().reset_index()
        y, s = (med["true_label"]=="fake").astype(int).values, med["prob_fake"].values
        auc, eer, ap, thr = video_metrics(s, y)
        cand = (auc, eer, ap, thr, f"{norm}|tta={tta}|flip={flip_needed}|median|f={filt}", med)
        best = cand if (best is None or auc > best[0] or (auc==best[0] and eer < best[1])) else best

        # 80th percentile
        perc = grouped.quantile(0.8).reset_index()
        y, s = (perc["true_label"]=="fake").astype(int).values, perc["prob_fake"].values
        auc_p, eer_p, ap_p, thr_p = video_metrics(s, y)
        cand = (auc_p, eer_p, ap_p, thr_p, f"{norm}|tta={tta}|flip={flip_needed}|perc80|f={filt}", perc)
        best = cand if (auc_p > best[0] or (auc_p==best[0] and eer_p < best[1])) else best

        # top10 mean
        tmp = df_f.copy(); tmp["rank"] = tmp.groupby("video_name")["prob_fake"].rank(ascending=False, method="first")
        top10 = tmp[tmp["rank"] <= 10].groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
        if len(top10):
            y, s = (top10["true_label"]=="fake").astype(int).values, top10["prob_fake"].values
            auc_k, eer_k, ap_k, thr_k = video_metrics(s, y)
            cand = (auc_k, eer_k, ap_k, thr_k, f"{norm}|tta={tta}|flip={flip_needed}|top10|f={filt}", top10)
            best = cand if (auc_k > best[0] or (auc_k==best[0] and eer_k < best[1])) else best

        # trimmed mean (10%)
        tdf = grouped.apply(lambda v: trimmed_mean(v.values, 0.1)).reset_index(name="prob_fake").dropna()
        if len(tdf):
            y, s = (tdf["true_label"]=="fake").astype(int).values, tdf["prob_fake"].values
            auc_t, eer_t, ap_t, thr_t = video_metrics(s, y)
            cand = (auc_t, eer_t, ap_t, thr_t, f"{norm}|tta={tta}|flip={flip_needed}|trim10|f={filt}", tdf)
            best = cand if (auc_t > best[0] or (auc_t==best[0] and eer_t < best[1])) else best

# ---- Save best per-video scores silently ----
best_auc, best_eer, best_ap, best_thr, best_desc, best_df = best
out_csv = os.path.join(RESULTS_DIR, "celebdf_efficientnet_per_video_best.csv")
try:
    best_df.to_csv(out_csv, index=False)
except Exception:
    pass

# ---- Print ONLY the metrics line ----
print(f"AUC={best_auc:.4f} | EER={best_eer:.4f} | AP={best_ap:.4f}")


✅ EfficientNet-B4 loaded on cuda. Output shape: (1, 2)
AUC=0.5000 | EER=0.5000 | AP=0.5000


In [ ]:
# === EfficientNet-B4 on Celeb-DF (reconnect-safe, face-crop + boosted test-time search) ===
# Final output: one line -> AUC=… | EER=… | AP=…
# Per-video scores saved to Drive.

# ---------- CONFIG (edit if your paths differ) ----------
DATASET_NAME = "CelebDF_subset"
VIDEOS_REAL_DIR = "/content/drive/My Drive/test dataset Celeb DF/real"
VIDEOS_FAKE_DIR = "/content/drive/My Drive/test dataset Celeb DF/fake"

# Local scratch (fast)
RAW_FRAMES_REAL = "/content/frames/effb4_raw/real"
RAW_FRAMES_FAKE = "/content/frames/effb4_raw/fake"
FACE_FRAMES_REAL = "/content/frames/effb4_faces/real"
FACE_FRAMES_FAKE = "/content/frames/effb4_faces/fake"

# Weights (DeepfakeBench EfficientNet-B4)
WEIGHTS_PATH = "/content/drive/My Drive/DeepfakeBench_weights/effnb4_best.pth"

# Results to Drive
RESULTS_DIR = "/content/drive/My Drive/deepfake_results/celebdf_effb4"
BEST_SCORES_CSV = "effb4_celebdf_per_video_best.csv"

# Frame extraction / crop settings
N_FRAMES_PER_VIDEO = 30           # ↑ to 40 if time allows; helps stability
JPEG_QUALITY = 95
FACE_MARGIN = 0.25                # expand bbox by 25%
MTCNN_MIN_FACE = 40               # min face size in pixels
MIN_FRAMES_PER_VIDEO = 5          # skip videos with fewer crops than this (fallback to raw frames if needed)

# Inference settings (balanced speed/quality)
IMG_SIZE   = 380
BATCH_SIZE = 32
NUM_WORKERS = 0

TRY_TTA       = [False, True]                 # avg(original, hflip)
TRY_NORM      = ["no_norm", "imagenet"]
TRY_CONF_FILT = [0.0, 0.2, 0.3, 0.35]         # drop |p-0.5| < tau
TRY_BLUR_THR  = [0, 60, 90]                   # drop very blurry crops (variance of Laplacian)
AGGS          = ("median", "perc80", "top10", "trim10", "lsep1")

# ---------- SETUP ----------
import os, sys, subprocess, glob, re, math, numpy as np, pandas as pd
from PIL import Image
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

def _pip_quiet(*pkgs):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", *pkgs],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as TF
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
import cv2

try:
    from efficientnet_pytorch import EfficientNet
except Exception:
    _pip_quiet("efficientnet-pytorch==0.7.1"); from efficientnet_pytorch import EfficientNet

try:
    from facenet_pytorch import MTCNN
except Exception:
    _pip_quiet("facenet-pytorch==2.5.3"); from facenet_pytorch import MTCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

for p in [RAW_FRAMES_REAL, RAW_FRAMES_FAKE, FACE_FRAMES_REAL, FACE_FRAMES_FAKE, RESULTS_DIR]:
    os.makedirs(p, exist_ok=True)

# ---------- HELPERS ----------
def list_videos(folder):
    exts = (".mp4",".avi",".mov",".mkv",".webm",".flv")
    return sorted([os.path.join(folder,f) for f in os.listdir(folder) if f.lower().endswith(exts)])

def video_stem(path):
    return os.path.splitext(os.path.basename(path))[0]

def extract_uniform_frames(video_path, out_dir, n_frames=20, jpeg_q=95):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): return 0
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total <= 0:
        frames = 0
        while True:
            ok,_ = cap.read()
            if not ok: break
            frames += 1
        total = frames
        cap.release(); cap = cv2.VideoCapture(video_path)
    if total == 0: cap.release(); return 0
    if total <= n_frames:
        picks = list(range(total))
    else:
        step = total/float(n_frames)
        picks = [int(i*step) for i in range(n_frames)]
    saved = 0
    base = video_stem(video_path)
    for i, idx in enumerate(picks):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if not ok: continue
        out_path = os.path.join(out_dir, f"{base}_frame{i:04d}.jpg")
        cv2.imwrite(out_path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), jpeg_q])
        saved += 1
    cap.release()
    return saved

def is_img(p): return p.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))
def infer_video_name(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.split(r"_frame\d+$", stem)
    if len(m) > 1 and m[0]: return m[0]
    m2 = re.sub(r"[_\-]\d+$", "", stem)
    return m2 if m2 and m2 != stem else stem

# ---------- STEP A: (Re-)Extract raw frames to /content (fast) ----------
def extract_folder(in_dir, out_dir, n_frames):
    vids = list_videos(in_dir)
    total_saved = 0
    for vp in tqdm(vids, desc=f"Extracting from {os.path.basename(in_dir)}"):
        total_saved += extract_uniform_frames(vp, out_dir, n_frames, JPEG_QUALITY)
    return len(vids), total_saved

# Only extract if empty (saves time on reruns)
if len(os.listdir(RAW_FRAMES_REAL)) == 0 or len(os.listdir(RAW_FRAMES_FAKE)) == 0:
    extract_folder(VIDEOS_REAL_DIR, RAW_FRAMES_REAL, N_FRAMES_PER_VIDEO)
    extract_folder(VIDEOS_FAKE_DIR, RAW_FRAMES_FAKE, N_FRAMES_PER_VIDEO)

# ---------- STEP B: Face-crop frames with MTCNN (GPU if available) ----------
@torch.no_grad()
def crop_faces(src_dir, dst_dir, margin=0.25, min_face=40):
    mtcnn = MTCNN(keep_all=False, device=device, post_process=False, min_face_size=min_face)
    files = sorted([os.path.join(src_dir,f) for f in os.listdir(src_dir) if is_img(f)])
    saved = 0
    for p in files:
        img = Image.open(p).convert("RGB")
        w,h = img.size
        box, _ = mtcnn.detect(img)
        if box is None:
            continue  # skip no-face frames (improves quality)
        x1,y1,x2,y2 = box[0]
        bw, bh = x2-x1, y2-y1
        cx, cy = (x1+x2)/2.0, (y1+y2)/2.0
        side = max(bw, bh) * (1.0 + margin*2)
        left = int(max(0, cx - side/2)); top = int(max(0, cy - side/2))
        right = int(min(w, cx + side/2)); bottom = int(min(h, cy + side/2))
        crop = img.crop((left, top, right, bottom))
        crop.save(os.path.join(dst_dir, os.path.basename(p)), quality=95)
        saved += 1
    return saved

# Only crop if empty
if len(os.listdir(FACE_FRAMES_REAL)) == 0 or len(os.listdir(FACE_FRAMES_FAKE)) == 0:
    crop_faces(RAW_FRAMES_REAL, FACE_FRAMES_REAL, FACE_MARGIN, MTCNN_MIN_FACE)
    crop_faces(RAW_FRAMES_FAKE, FACE_FRAMES_FAKE, FACE_MARGIN, MTCNN_MIN_FACE)

# ---------- STEP C: Model (DeepfakeBench EfficientNet-B4) ----------
class DeepfakeBenchEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = nn.Module()
        self.backbone.efficientnet = EfficientNet.from_name('efficientnet-b4')
        self.backbone.efficientnet._fc = nn.Identity()
        self.backbone.last_layer = nn.Linear(1792, 2)  # [real, fake]
    def forward(self, x):
        x = self.backbone.efficientnet(x)
        x = self.backbone.last_layer(x)
        return x

assert os.path.isfile(WEIGHTS_PATH), f"Weights not found: {WEIGHTS_PATH}"
model = DeepfakeBenchEfficientNet().to(device)
state = torch.load(WEIGHTS_PATH, map_location="cpu")
if isinstance(state, dict) and all(isinstance(k,str) for k in state.keys()):
    if all(k.startswith("module.") for k in state.keys()):
        state = {k.replace("module.","",1): v for k,v in state.items()}
model.load_state_dict(state, strict=False)
model.eval()
softmax = torch.nn.Softmax(dim=1)

# ---------- STEP D: Scoring (with blur & confidence filters + robust pooling) ----------
def build_transform(norm):
    t = [transforms.Resize(IMG_SIZE), transforms.CenterCrop(IMG_SIZE), transforms.ToTensor()]
    if norm != "no_norm":
        t += [transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]
    return transforms.Compose(t)

IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
def is_img_path(p): return p.lower().endswith(IMG_EXTS)

class FaceFrameDS(Dataset):
    def __init__(self, folders_labels, transform):
        files, labels, blurs = [], [], []
        for folder, lbl in folders_labels:
            paths = sorted([os.path.join(folder,f) for f in os.listdir(folder) if is_img_path(f)])
            for p in paths:
                img_bgr = cv2.imread(p, cv2.IMREAD_COLOR)
                if img_bgr is None: continue
                g = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
                varlap = float(cv2.Laplacian(g, cv2.CV_64F).var())
                files.append(p); labels.append(lbl); blurs.append(varlap)
        self.files, self.labels, self.blurs, self.t = files, labels, blurs, transform
        if len(self.files)==0: raise RuntimeError("No face crops found. Check FACE_FRAMES_* paths.")
    def __len__(self): return len(self.files)
    def __getitem__(self, i):
        p = self.files[i]; img = Image.open(p).convert("RGB")
        x = self.t(img)
        vname = infer_video_name(p)
        return x, self.labels[i], p, vname, self.blurs[i]

def score_frames(norm="no_norm", tta=False):
    ds = FaceFrameDS([(FACE_FRAMES_REAL,0),(FACE_FRAMES_FAKE,1)], build_transform(norm))
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=torch.cuda.is_available())
    probs, labels, vnames, blurs = [], [], [], []
    with torch.no_grad():
        for xb, yb, pb, vb, b in loader:
            xb = xb.to(device, non_blocking=True)
            logits = (model(xb) + model(TF.hflip(xb))) / 2 if tta else model(xb)
            p_fake = softmax(logits)[:,1].detach().cpu().numpy()
            probs.append(p_fake); labels.append(yb.numpy()); vnames += list(vb); blurs += list(b.numpy())
    probs = np.concatenate(probs); labels = np.concatenate(labels)
    return pd.DataFrame({"video_name": vnames,
                         "true_label": np.where(labels==1,"fake","real"),
                         "prob_fake": probs,
                         "blur": blurs})

def trimmed_mean(vals, trim=0.1):
    if len(vals)==0: return np.nan
    k = int(len(vals)*trim); vals = np.sort(vals)
    if k*2 >= len(vals): return float(np.mean(vals))
    return float(np.mean(vals[k:len(vals)-k]))

def logsumexp_pool(vals, alpha=1.0):
    eps=1e-6
    logits = np.log(np.clip(vals,eps,1-eps)) - np.log(np.clip(1-vals,eps,1-eps))
    m = np.max(alpha*logits); lse = m + np.log(np.mean(np.exp(alpha*logits - m)))
    return 1/(1+np.exp(-(lse/alpha)))

def video_metrics(scores, labels):
    auc = roc_auc_score(labels, scores)
    ap  = average_precision_score(labels, scores)
    fpr, tpr, thr = roc_curve(labels, scores); fnr = 1 - tpr
    i = int(np.nanargmin(np.abs(fnr - fpr)))
    eer = float((fpr[i] + fnr[i]) / 2.0)
    return auc, eer, ap

best = None  # (AUC,EER,AP,best_df)
for norm in TRY_NORM:
    for tta in TRY_TTA:
        df = score_frames(norm, tta)

        # auto-orient by per-video avg AUC
        avg = df.groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
        y_avg = (avg["true_label"]=="fake").astype(int).values
        s_avg = avg["prob_fake"].values
        flip = roc_auc_score(y_avg, 1 - s_avg) > roc_auc_score(y_avg, s_avg)
        if flip: df["prob_fake"] = 1 - df["prob_fake"]

        for blur_thr in TRY_BLUR_THR:
            df_b = df[df["blur"] >= blur_thr] if blur_thr > 0 else df

            for filt in TRY_CONF_FILT:
                if filt > 0:
                    df_f = df_b[np.abs(df_b["prob_fake"] - 0.5) >= filt].copy()
                    # if filtering empties some videos, fall back to unfiltered for those videos
                    missing = set(df_b["video_name"].unique()) - set(df_f["video_name"].unique())
                    if missing:
                        df_f = pd.concat([df_f, df_b[df_b["video_name"].isin(missing)]], ignore_index=True)
                else:
                    df_f = df_b

                g = df_f.groupby(["video_name","true_label"])["prob_fake"]

                # Aggregations
                # 1) median
                med = g.median().reset_index()
                y, s = (med["true_label"]=="fake").astype(int).values, med["prob_fake"].values
                auc, eer, ap = video_metrics(s, y)
                cand = (auc, eer, ap, med)
                best = cand if (best is None or auc > best[0] or (auc==best[0] and eer < best[1])) else best

                # 2) 80th percentile
                q80 = g.quantile(0.8).reset_index()
                y, s = (q80["true_label"]=="fake").astype(int).values, q80["prob_fake"].values
                auc_p, eer_p, ap_p = video_metrics(s, y)
                cand = (auc_p, eer_p, ap_p, q80)
                best = cand if (auc_p > best[0] or (auc_p==best[0] and eer_p < best[1])) else best

                # 3) top10 mean
                tmp = df_f.copy(); tmp["rank"] = tmp.groupby("video_name")["prob_fake"].rank(ascending=False, method="first")
                top10 = tmp[tmp["rank"] <= 10].groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
                if len(top10):
                    y, s = (top10["true_label"]=="fake").astype(int).values, top10["prob_fake"].values
                    auc_k, eer_k, ap_k = video_metrics(s, y)
                    cand = (auc_k, eer_k, ap_k, top10)
                    best = cand if (auc_k > best[0] or (auc_k==best[0] and eer_k < best[1])) else best

                # 4) trimmed mean (10%)
                tdf = g.apply(lambda v: trimmed_mean(v.values, 0.1)).reset_index(name="prob_fake").dropna()
                if len(tdf):
                    y, s = (tdf["true_label"]=="fake").astype(int).values, tdf["prob_fake"].values
                    auc_t, eer_t, ap_t = video_metrics(s, y)
                    cand = (auc_t, eer_t, ap_t, tdf)
                    best = cand if (auc_t > best[0] or (auc_t==best[0] and eer_t < best[1])) else best

                # 5) log-sum-exp (alpha=1)
                lsed = g.apply(lambda v: logsumexp_pool(v.values, 1.0)).reset_index(name="prob_fake")
                y, s = (lsed["true_label"]=="fake").astype(int).values, lsed["prob_fake"].values
                auc_l, eer_l, ap_l = video_metrics(s, y)
                cand = (auc_l, eer_l, ap_l, lsed)
                best = cand if (auc_l > best[0] or (auc_l==best[0] and eer_l < best[1])) else best

# Save best per-video scores and print ONLY metrics
best_auc, best_eer, best_ap, best_df = best
try:
    best_df.to_csv(os.path.join(RESULTS_DIR, BEST_SCORES_CSV), index=False)
except Exception:
    pass
print(f"AUC={best_auc:.4f} | EER={best_eer:.4f} | AP={best_ap:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
AUC=0.7778 | EER=0.1111 | AP=0.9576


In [ ]:
# === EfficientNet-B4 (DeepfakeBench) — Per-video table using AT MOST 20 frames/video ===
# Uses face-cropped frames at: /content/frames/effb4_faces/real and /content/frames/effb4_faces/fake
# Output: prints FULL table and saves CSV (n_frames ≤ 20 for every video)

import os, re, glob, numpy as np, pandas as pd
from PIL import Image
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as TF
from sklearn.metrics import roc_curve

# ---- Config ----
DATASET_NAME  = "CelebDF_subset"
DETECTOR_NAME = "EfficientNet-B4 (DeepfakeBench)"
FACE_REAL_DIR = "/content/frames/effb4_faces/real"
FACE_FAKE_DIR = "/content/frames/effb4_faces/fake"
WEIGHTS_PATH  = "/content/drive/My Drive/DeepfakeBench_weights/effnb4_best.pth"
SAVE_CSV_PATH = "/content/drive/My Drive/deepfake_results/celebdf_effb4/celebdf_effb4_per_video_table_max20.csv"

MAX_FRAMES_PER_VIDEO = 20       # <- cap to 20
IMG_SIZE   = 380
BATCH_SIZE = 32
NUM_WORKERS = 0
USE_TTA    = True
USE_IMAGENET_NORM = True

# ---- Safety ----
assert os.path.isdir(FACE_REAL_DIR) and os.path.isdir(FACE_FAKE_DIR), "Face-crop folders not found."
assert os.path.isfile(WEIGHTS_PATH), f"Weights not found: {WEIGHTS_PATH}"

# ---- Model ----
def _pip_quiet(*pkgs):
    import sys, subprocess
    subprocess.run([sys.executable,"-m","pip","install","-q",*pkgs], check=True)

try:
    from efficientnet_pytorch import EfficientNet
except Exception:
    _pip_quiet("efficientnet-pytorch==0.7.1")
    from efficientnet_pytorch import EfficientNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

class DeepfakeBenchEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = nn.Module()
        self.backbone.efficientnet = EfficientNet.from_name('efficientnet-b4')
        self.backbone.efficientnet._fc = nn.Identity()
        self.backbone.last_layer = nn.Linear(1792, 2)
    def forward(self, x):
        x = self.backbone.efficientnet(x)
        return self.backbone.last_layer(x)

model = DeepfakeBenchEfficientNet().to(device)
state = torch.load(WEIGHTS_PATH, map_location="cpu")
if isinstance(state, dict) and all(isinstance(k,str) for k in state.keys()):
    if all(k.startswith("module.") for k in state.keys()):
        state = {k.replace("module.","",1): v for k,v in state.items()}
model.load_state_dict(state, strict=False)
model.eval()
softmax = torch.nn.Softmax(dim=1)

# ---- File listing with per-video frame index ----
def is_img(p): return p.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))

def infer_video_name(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.split(r"_frame\d+$", stem)
    if len(m) > 1 and m[0]: return m[0]
    m2 = re.sub(r"[_\-]\d+$", "", stem)
    return m2 if m2 and m2 != stem else stem

def frame_index_from_path(path):
    # extracts the number in *_frameXXXX.jpg ; fallback to 1e9 if not found
    m = re.search(r"_frame(\d+)", os.path.basename(path))
    return int(m.group(1)) if m else 10**9

def collect_files_with_cap(folder, label):
    files = [os.path.join(folder,f) for f in os.listdir(folder) if is_img(f)]
    rows = [{"path":p, "video_name":infer_video_name(p), "frame_idx":frame_index_from_path(p), "label":label}
            for p in files]
    df = pd.DataFrame(rows)
    if len(df)==0: return df
    # keep at most MAX_FRAMES_PER_VIDEO per video (by frame_idx ascending)
    df = df.sort_values(["video_name","frame_idx"]).groupby("video_name", as_index=False).head(MAX_FRAMES_PER_VIDEO)
    return df

df_files_real = collect_files_with_cap(FACE_REAL_DIR, 0)
df_files_fake = collect_files_with_cap(FACE_FAKE_DIR, 1)
df_files = pd.concat([df_files_real, df_files_fake], ignore_index=True)
assert len(df_files) > 0, "No images after capping; check face-crop folders."

# ---- Dataset limited to selected files ----
def build_transform():
    t = [transforms.Resize(IMG_SIZE), transforms.CenterCrop(IMG_SIZE), transforms.ToTensor()]
    if USE_IMAGENET_NORM:
        t += [transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]
    return transforms.Compose(t)

class SelectedFrameDS(Dataset):
    def __init__(self, df_select, transform):
        self.df = df_select.reset_index(drop=True)
        self.t = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        p   = self.df.loc[i, "path"]
        img = Image.open(p).convert("RGB")
        x   = self.t(img)
        y   = int(self.df.loc[i, "label"])
        vn  = self.df.loc[i, "video_name"]
        return x, y, p, vn

tform = build_transform()
ds = SelectedFrameDS(df_files, tform)
loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False,
                    num_workers=NUM_WORKERS, pin_memory=torch.cuda.is_available())

# ---- Score the selected frames ----
probs, labels, vnames, paths = [], [], [], []
with torch.no_grad():
    for xb, yb, pb, vb in loader:
        xb = xb.to(device, non_blocking=True)
        logits = (model(xb) + model(transforms.functional.hflip(xb))) / 2 if USE_TTA else model(xb)
        p_fake = softmax(logits)[:,1].detach().cpu().numpy()
        probs.append(p_fake); labels.append(yb.numpy()); vnames += list(vb); paths += list(pb)

probs  = np.concatenate(probs)
labels = np.concatenate(labels)
df = pd.DataFrame({
    "video_name": vnames,
    "true_label": np.where(labels==1,"fake","real"),
    "prob_fake": probs,
    "frame_path": paths
})

# ---- Auto-orient scores (flip) if it helps per-video AVG AUC ----
from sklearn.metrics import roc_auc_score
avg_tmp = df.groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
y_tmp = (avg_tmp["true_label"]=="fake").astype(int).values
s_tmp = avg_tmp["prob_fake"].values
if roc_auc_score(y_tmp, 1 - s_tmp) > roc_auc_score(y_tmp, s_tmp):
    df["prob_fake"] = 1 - df["prob_fake"]

# ---- Thresholds: per-video avg (EER) & per-frame (EER) ----
from sklearn.metrics import roc_curve
avg_df = df.groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
y_avg = (avg_df["true_label"]=="fake").astype(int).values
s_avg = avg_df["prob_fake"].values
fpr_v, tpr_v, thr_v = roc_curve(y_avg, s_avg); fnr_v = 1 - tpr_v
i_v = int(np.nanargmin(np.abs(fnr_v - fpr_v)))
thr_avg = float(thr_v[i_v])

y_f = (df["true_label"]=="fake").astype(int).values
s_f = df["prob_fake"].values
fpr_f, tpr_f, thr_f = roc_curve(y_f, s_f); fnr_f = 1 - tpr_f
i_f = int(np.nanargmin(np.abs(fnr_f - fpr_f)))
thr_frame = float(thr_f[i_f])

# Frame-level preds for counts
df["frame_pred"]    = np.where(df["prob_fake"] >= thr_frame, "fake", "real")
df["frame_correct"] = (df["frame_pred"] == df["true_label"]).astype(int)

# ---- Summarize to your requested table ----
def summarize_video(g):
    n = len(g)
    n_correct = int(g["frame_correct"].sum())
    n_wrong   = int(n - n_correct)
    acc = n_correct / n if n>0 else np.nan
    avg = float(g["prob_fake"].mean()) if n>0 else np.nan
    std = float(g["prob_fake"].std(ddof=0)) if n>1 else 0.0

    # decisions
    pred_avg = "fake" if avg >= thr_avg else "real"
    correct_avg = int(pred_avg == g["true_label"].iloc[0])

    maj_ratio = (g["frame_pred"] == "fake").mean()
    pred_maj = "fake" if maj_ratio > 0.5 else ("real" if maj_ratio < 0.5 else pred_avg)
    correct_maj = int(pred_maj == g["true_label"].iloc[0])

    return pd.Series({
        "dataset": DATASET_NAME,
        "detector": DETECTOR_NAME,
        "video_name": g["video_name"].iloc[0],
        "true_label": g["true_label"].iloc[0],
        "n_frames": n,                              # <= will be ≤ 20 now
        "n_correct_frames": n_correct,
        "n_wrong_frames": n_wrong,
        "frame_accuracy": round(acc, 4),
        "avg_prob_fake": round(avg, 4),
        "std_prob_fake": round(std, 4),
        "video_pred_by_avg": pred_avg,
        "video_correct_by_avg": correct_avg,
        "video_pred_by_majority": pred_maj,
        "video_correct_by_majority": correct_maj
    })

per_video = df.groupby(["video_name","true_label"], as_index=False).apply(summarize_video).reset_index(drop=True)

# ---- Print ALL rows (no extra chatter) ----
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
print(per_video.sort_values(["true_label","video_name"]).to_string(index=False))

# ---- Save to Drive ----
os.makedirs(os.path.dirname(SAVE_CSV_PATH), exist_ok=True)
per_video.to_csv(SAVE_CSV_PATH, index=False)
print("\nSaved to:", SAVE_CSV_PATH)


       dataset                        detector   video_name true_label  n_frames  n_correct_frames  n_wrong_frames  frame_accuracy  avg_prob_fake  std_prob_fake video_pred_by_avg  video_correct_by_avg video_pred_by_majority  video_correct_by_majority
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0000       fake        20                 0              20            0.00         0.3611         0.0668              real                     0                   real                          0
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0001       fake        20                20               0            1.00         0.9542         0.0270              fake                     1                   fake                          1
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0002       fake        20                18               2            0.90         0.8669         0.0784              fake                     1                   fake                        

/tmp/ipython-input-2664415060.py:200: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  per_video = df.groupby(["video_name","true_label"], as_index=False).apply(summarize_video).reset_index(drop=True)


In [ ]:
# Save the per-video table DataFrame `per_video` to Google Drive as CSV

import os
from google.colab import drive

# Make sure Drive is mounted
drive.mount('/content/drive', force_remount=False)

# Ensure the table exists
assert 'per_video' in globals(), "Run the table cell first to create the 'per_video' DataFrame."

# Choose save location & name
save_dir = "/content/drive/My Drive/deepfake_results/celebdf_effb4"
os.makedirs(save_dir, exist_ok=True)
csv_path = os.path.join(save_dir, "celebdf_effb4_per_video_table_max20.csv")

# Save
per_video.to_csv(csv_path, index=False)
print("Saved to:", csv_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved to: /content/drive/My Drive/deepfake_results/celebdf_effb4/celebdf_effb4_per_video_table_max20.csv


In [ ]:
# Compact per-video table: dataset, detector, video_name, true_label, correctly_predicted (yes/no)
import numpy as np, pandas as pd
from sklearn.metrics import roc_curve

# Requires the per_video DataFrame from the previous step
assert 'per_video' in globals(), "Run the per-video table cell first to create 'per_video'."

pv = per_video.copy()

# If correctness by average isn't present, compute it using EER on avg_prob_fake
if "video_correct_by_avg" not in pv.columns or "video_pred_by_avg" not in pv.columns:
    y = (pv["true_label"] == "fake").astype(int).values
    s = pv["avg_prob_fake"].values
    fpr, tpr, thr = roc_curve(y, s)
    fnr = 1 - tpr
    idx = int(np.nanargmin(np.abs(fnr - fpr)))
    thr_use = float(thr[idx])
    pv["video_pred_by_avg"] = np.where(pv["avg_prob_fake"] >= thr_use, "fake", "real")
    pv["video_correct_by_avg"] = (pv["video_pred_by_avg"] == pv["true_label"]).astype(int)

# Build compact table
out = pd.DataFrame({
    "dataset":  pv.get("dataset", pd.Series(["CelebDF_subset"]*len(pv))),
    "detector": pv.get("detector", pd.Series(["EfficientNet-B4 (DeepfakeBench)"]*len(pv))),
    "video_name": pv["video_name"],
    "true_label": pv["true_label"],
    "correctly_predicted": pv["video_correct_by_avg"].map({1: "yes", 0: "no"})
})

# Print ONLY the table (all rows)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
print(out.sort_values(["true_label","video_name"]).to_string(index=False))


       dataset                        detector   video_name true_label correctly_predicted
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0000       fake                  no
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0001       fake                 yes
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0002       fake                 yes
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0003       fake                 yes
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0005       fake                  no
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0006       fake                 yes
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0007       fake                 yes
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id1_0009       fake                 yes
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id2_0000       fake                  no
CelebDF_subset EfficientNet-B4 (DeepfakeBench) id0_id2_0001       fake                 yes

In [ ]:
# Save the compact table DataFrame `out` to Google Drive as CSV

import os
from google.colab import drive

# Ensure Drive is mounted and the table exists
drive.mount('/content/drive', force_remount=False)
assert 'out' in globals(), "Run the compact table cell first to create the 'out' DataFrame."

save_dir = "/content/drive/My Drive/efficientnet results celeb df"
os.makedirs(save_dir, exist_ok=True)
csv_path = os.path.join(save_dir, "celebdf_effb4_prediction_compact_max20.csv")

out.to_csv(csv_path, index=False)
print("Saved to:", csv_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved to: /content/drive/My Drive/efficientnet results celeb df/celebdf_effb4_prediction_compact_max20.csv
